# Natural Language Inference (NLI) Results and Error Analysis

In this Jupyter Notebook, we will load a pretrained NLI model, demonstrate its usage on custom examples, present an overview of the results, and perform error analysis.

## Load a Pretrained Model

In [3]:
import torch
import models

config_nli_model = {
    'n_words'        :  37927         ,
    'word_emb_dim'   :  300   ,
    'enc_lstm_dim'   :  2048   ,
    'dpout_model'    :  0.0    ,
    'fc_dim'         :  512         ,
    'bsize'          :  16     ,
    'n_classes'      :  3      ,
    'encoder_type'   :  'LSTMEncoder'   ,
    'use_cuda'       :  True                  ,
    }

model_path = "savedir/lstm_model.pickle"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
state_dict = torch.load(model_path, map_location=device)

# Create a model instance and load the state dictionary
model_instance = models.NLIClassifier(config_nli_model)  # Assuming the LSTMEncoder is imported from models module
model_instance.load_state_dict(state_dict)
model_instance.to(device)
model_instance.eval()

NLIClassifier(
  (encoder): LSTMEncoder(
    (enc_lstm): LSTM(300, 2048)
  )
  (classifier): Sequential(
    (0): Linear(in_features=8192, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=3, bias=True)
  )
)

## Demonstrate Model Usage

In [21]:
from data_preprocess import NLIDataset, build_vocab, get_nli, collate_fn
import numpy as np

LABELS = ['entailment', 'neutral', 'contradiction']

def predict_entailment(model, premise, hypothesis):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    data_path = "data/"
    glove_path = "glove/glove.840B.300d.txt"

    train_data, valid_data, test_data = get_nli(data_path)
    word_vec = build_vocab(train_data['s1'] + train_data['s2'], glove_path)
    dataset = NLIDataset(train_data, word_vec)

    s1_idx, s1_length = dataset._get_sentence_indices(premise)
    s2_idx, s2_length = dataset._get_sentence_indices(hypothesis)
    s1_tensor, s1_length = s1_idx.unsqueeze(0).to(device), torch.tensor([s1_length]).to("cpu")
    s2_tensor, s2_length = s2_idx.unsqueeze(0).to(device), torch.tensor([s2_length]).to("cpu")
    s1 = (s1_tensor, s1_length)
    s2 = (s2_tensor, s2_length)

    logits = model(s1, s2)
    prediction = np.argmax(logits.detach().cpu().numpy(), axis=1)

    return prediction


In [22]:
premise = "Two men sitting in the sun"
hypothesis = "Nobody is sitting in the shade"

prediction = predict_entailment(model_instance, premise, hypothesis)
print("Prediction:", LABELS[prediction[0]])

Found 37325/62999 words with glove vectors
Vocab size : 37325
Prediction: entailment
